# POPULATION ANALYSIS USING SQL

### Aim: To analyze the common factors Affecting Population and see which has the highest bearing

In [1]:
#import necessary libraries
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Create helper functions to aid analysis

db = 'factbook.db'
def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)
    
def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)
    
def show_table():
    q = '''SELECT name,
                  type
           FROM sqlite_master
           WHERE type IN ('table', 'view')
            '''
    return run_query(q)

In [3]:
show_table()

,name,type


In [4]:
q = '''SELECT * FROM facts
        '''
run_query(q)

DatabaseError: Execution failed on sql 'SELECT * FROM facts
        ': no such table: facts

#### The dataset is a table of 13 columns and 261 rows showing various characteristics of individual countries

In [ ]:
data = run_query(q)
data.info()

In [ ]:
data.head()

In [ ]:
#Select countries with the top 20 population count, excluding the 'world' and 'European Union' poulation as these are not countries
q_1 = '''SELECT *
       FROM facts
       WHERE name != 'World' AND name != 'European Union'
       ORDER BY population DESC
       LIMIT 20
        '''
top_20_populated_countries = run_query(q_1)
top_20_populated_countries

In [ ]:
#Create and show a new colum, ppm, which is the population in millions
q_2 = '''SELECT name, 
                population, 
                (population / 1000000) ppm
         FROM facts
         WHERE name != 'World' AND name != 'European Union'
         ORDER BY population DESC
         LIMIT 10
            '''
top_10_populated_countries = run_query(q_2)
top_10_populated_countries

### Cleaned this dataset by removing countries in the top ten that are not actually countries. These are 'world' and 'European Union', i guess they can be attributed to human error as these are not countries

In [ ]:
top_10_populated_countries.plot(kind = 'bar',
                                x = 'name',
                                y = 'ppm',
                                rot = 30,
                                legend = False,
                                colormap = plt.cm.BuGn_r,
                                figsize = (15,5)
                                )
plt.ylabel('Population in Millions')
plt.xlabel('Country')
plt.title('Bar Plot Of The Top Ten Countries in the World', y = 1.07)

In [ ]:
top_10_populated_countries[['name', 'ppm']].plot(kind = 'pie',
                                            y = 'ppm',
                                            autopct = '%.2f%%',
                                            figsize = (8,8),
                                            legend = False,
                                            labels= ['China','India','United States','Indonesia','Brazil','Pakistan','Nigeria','Bangladesh','Russia','Japan']
                                                )
plt.title('Pie Chart Showing the Percentage of top 10 Populated Countries', loc = 'right')
plt.tight_layout()
plt.ylabel('')

### From the plots above we see China and India have at least four times the population of the remaining top 8 countries which is Amazing

In [ ]:
q_3 = '''SELECT name country,
                population,
                population_growth,
                birth_rate,
                death_rate,
                migration_rate
         FROM facts
         WHERE name != 'World' AND name!= 'European Union'
         ORDER BY population DESC
         LIMIT 10
            '''
run_query(q_3)

Decided to write this query first to aid the reader in understanding how the more complex query below was written

In [ ]:
q_4 = ''' SELECT * FROM (SELECT name country,
                population,
                population_growth,
                birth_rate,
                death_rate,
                migration_rate,
                population / 1000000 ppm
         FROM facts
         WHERE name != 'World' AND name!= 'European Union' AND ppm > 0
         ORDER BY population DESC
         LIMIT 100)
           ORDER BY ppm ASC
            '''
run_query(q_4).head(15)

### The data was 'segmented' in such a way as to cater for countries that were wrongly inputed as countries. Examples are European Union and World. I also used the condition of removing countries with population(ppm) less than one million.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))
ax1, ax2, ax3, ax4 = axes.flatten()
fig.subplots_adjust(hspace=.5, wspace=.3)

#top left
run_query(q_4).plot(kind = 'scatter',
                    ax = ax1,
                    x = 'migration_rate',
                    y = 'ppm',
                   title = 'Scatter Plot of Migration_Rate Vs PPM')

#top right
run_query(q_4).plot(kind = 'scatter',
                    ax = ax2,
                    x = 'population_growth',
                    y = 'ppm',
                   title = 'Scatter Plot of Population_growth Vs PPM')

#Bottom left
run_query(q_4).plot(kind = 'scatter',
                    ax = ax3,
                    x = 'birth_rate',
                    y = 'ppm',
                   title = 'Scatter Plot of Birth_Rate Vs PPM')

#Bottom right
run_query(q_4).plot(kind = 'scatter',
                    ax = ax4,
                    x = 'death_rate',
                    y = 'ppm',
                   title = 'Scatter Plot of Death_Rate Vs PPM')

plt.show()

#### From the result above, there is a positive correlation between these factors(migration rate, birth rate, population growth and death rate) and the Population of a country. 

#### But, of all of these factors, the migration_rate stands out. The plot(top left) shows us that, majority of the nations with small population have low migration rate. We cannot draw any significant inference because we are not sure if these are immigrants or emigrants

In [5]:
#close connection
conn = sqlite3.connect(db)
conn.close()

# Conclusion

### We can see that migration rate according to the data provided is one factor that stands out among the factors affecting population in the world